In [1]:
library(R.matlab)
library(wavelets)
library(tuneR)
library(seewave)
library(data.table)
library(constellation)
library(GENEAread)

R.matlab v3.6.2 (2018-09-26) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen


Loading required package: bitops

Loading required package: mmap



In [ ]:
baseName <- "frame"
wavesName = c("8PSK", "16QAM", "64QAM", "BPSK", "CPFSK", "GFSK", "QPSK", "PAM4")
dffile <- data.frame(modulation=wavesName)

timeDomain_columns <- paste('timeDomain', c(1:1024), sep='_')
dffile[, timeDomain_columns] <- NA

fourierDomain_columns <- paste('fourierDomain', c(1:1024), sep='_')
dffile[, fourierDomain_columns] <- NA

waveletsDomain_columns <- c("wavelets_max_serie_1", "wavelets_min_serie_1" ,"wavelets_mean_serie_1", "wavelets_1st_Qu_serie_1" , "wavelets_2nd_Qu_serie_1", "wavelets_3rd_Qu_serie_1", 
                            "wavelets_max_serie_2", "wavelets_min_serie_2" ,"wavelets_mean_serie_2", "wavelets_1st_Qu_serie_2" , "wavelets_2nd_Qu_serie_2", "wavelets_3rd_Qu_serie_2", 
                            "wavelets_max_serie_3", "wavelets_min_serie_3" ,"wavelets_mean_serie_3", "wavelets_1st_Qu_serie_3" , "wavelets_2nd_Qu_serie_3", "wavelets_3rd_Qu_serie_3") 
dffile[, waveletsDomain_columns] <- NA

zeroCrossX_columns <- paste('zeroCross_X_', c(1:1024), sep='_')
dffile[, zeroCrossX_columns] <- NA

zeroCrossY_columns <- paste('zeroCross_Y_', c(1:1024), sep='_')
dffile[, zeroCrossY_columns] <- NA

autoCorrelation_columns <- paste('autoCorrelation_', c(1:1024), sep='_')
dffile[, autoCorrelation_columns] <- NA

cs_current_columns <- paste('cs_current_', c(1:1024), sep='_')
dffile[, cs_current_columns] <- NA

cs_prior_columns <- paste('cs_prior_', c(1:1024), sep='_')
dffile[, cs_prior_columns] <- NA

s_ww_columns <- c("sDomain_max", "sDomain_min" ,"sDomain_mean", "sDomain_1st_Qu" , "sDomain_2nd_Qu", "sDomain_3rd_Qu")
dffile[, s_ww_columns] <- NA
names_idx=1
row_idx = 1

for(i in 1:1000) {
    for (w in wavesName) {
        n <- formatC(i, width=3, flag="0")
        name <- paste(baseName, w, n, ".mat", sep="")
        pathfile <- paste("../dataset/origin/", name, sep="")
        signal <- readMat(pathfile)
        values <- signal['frame']$frame
        
        if(i > 1){
            dffile[row_idx, 'modulation'] <- wavesName[names_idx] 
            names_idx = names_idx + 1
            if(names_idx == 8) names_idx=1
        }
        
        #Time Domain
        t=seq(0,1023)
        s = Mod(values) * cos((pi*2.5*t) + Arg(values))
        dffile[row_idx, timeDomain_columns] <- s
        
        
        #Frequency Domain (Fourier)
        f_s <- fft(s)
        f_s_abs <- abs(f_s)
        dffile[row_idx, fourierDomain_columns] <- f_s_abs
        
        #Frequency Domain (Wavelets)
        s_wavelets <- dwt(s, filter="la8", n.levels=3, boundary="periodic", fast=TRUE)
        s_ww <- c(max(s_wavelets@W$W1), min(s_wavelets@W$W1), mean(s_wavelets@W$W1), quantile(s_wavelets@W$W1, probs = c(0.25, 0.5, 0.75), names=FALSE),
                   max(s_wavelets@W$W2), min(s_wavelets@W$W2), mean(s_wavelets@W$W2), quantile(s_wavelets@W$W2, probs = c(0.25, 0.5, 0.75), names=FALSE),
                   max(s_wavelets@W$W3), min(s_wavelets@W$W3), mean(s_wavelets@W$W3), quantile(s_wavelets@W$W3, probs = c(0.25, 0.5, 0.75), names=FALSE))
        dffile[row_idx, waveletsDomain_columns] <- s_ww
        
        #Zero-Cossing
        s_wave <- Wave(left=s, samp.rate=2500000000, bit=8)
        s_zc <- zc(s_wave, plot=FALSE)
        s_zc_x <- s_zc[,1]
        s_zc_y <- s_zc[,2]
        
        dffile[row_idx, zeroCrossX_columns] <- s_zc_x
        dffile[row_idx, zeroCrossY_columns] <- s_zc_y
        
        #autocorrelation
        ac <- acf(s, lag = 1023, plot=FALSE)
        dffile[row_idx, autoCorrelation_columns] <- ac$acf
        
        #Constellation Shape Features
        tt <- seq(from = as.POSIXct("00:00:00", format="%H:%M:%S", tz="UTC"), length.out = 1024, by = "mins")
        DF = data.table( ID = 1, TIME = tt, VALUE = s)
        DT_C_SHAPE <- value_change(DF, value = 1.5, direction = "up", window_hours = 18, join_key = "ID", time_var = "TIME", value_var = "VALUE.V1", mult = "all")
        cs_current <- DT_C_SHAPE$CURRENT_VALUE[c(1:1024)]
        cs_prior <- DT_C_SHAPE$PRIOR_VALUE[c(1:1024)]
        dffile[row_idx, cs_current_columns] <- cs_current
        dffile[row_idx, cs_prior_columns] <- cs_prior
        
        
        #S Transform
        s_s <- stft(s, plot = FALSE, reassign = TRUE, win = 100)
        s_s_vector <- as.vector(s_s[[1]])
        s_s_statista <- c(max(s_s_vector), min(s_s_vector), mean(s_s_vector), quantile(s_s_vector, probs = c(0.25, 0.5, 0.75), names=FALSE))
        dffile[row_idx, s_ww_columns] <- s_s_statista
        
        row_idx <- row_idx + 1 
    }
}

write.csv(dffile, "../dataset/transform/data.csv", row.names = FALSE)

In [ ]:
dfData <- read.csv("../dataset/transform/data100.csv")
dfData